# Lab 7 - Model Optimization for Inference

## 1. Evaluation mode - Exercise 1

#### Load the model and tokenize sample text

In [1]:
from transformers import AutoModel, AutoTokenizer

# load the model and tokenizer
model_name = "sentence-transformers/multi-qa-mpnet-base-cos-v1"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [2]:
# sample text from: https://wolnelektury.pl/katalog/lektura/lalka-tom-pierwszy.html

sample_text = """
Czuł w duszy dziwną pustkę, a na samym jej dnie coś, jakby kroplę piekącej goryczy. Żadnych sił, żadnych pragnień, nic, tylko tę kroplę tak małą, że jej niepodobna dojrzeć, a tak gorzką, że cały świat można by nią zatruć.

„Chwilowa apatia, wyczerpanie, brak wrażeń… Za dużo myślę o interesach” — mówił.
Stanął i patrzył. Dzień przedświąteczny i ładna pogoda wywabiły mnóstwo ludzi na bruk miejski. Sznur powozów i pstrokaty falujący tłum między Kopernikiem i Zygmuntem

wyglądał jak stado ptaków, które właśnie w tej chwili unosiły się nad miastem dążąc ku północy.

„Szczególna rzecz — mówił. — Każdy ptak w górze i każdy człowiek na ziemi wyobraża sobie, że idzie tam, dokąd chce. I dopiero ktoś stojący na boku widzi, że wszystkich razem pcha naprzód jakiś fatalny prąd, mocniejszy od ich przewidywań i pragnień. Może nawet ten sam, który unosi smugę iskier wydmuchniętych przez lokomotywę podczas nocy?… Błyszczą przez mgnienie oka, aby zgasnąć na całą wieczność, i to nazywa się życiem.

    Mijają ludzkie pokolenia
    Jak fale, gdy wiatr morzem zmąci;
    I nie masz godów ich pamięci,
    I nie masz bólów ich wspomnienia.

Gdzie ja to czytałem?… Wszystko jedno.”

„Oto miniatura kraju — myślał — w którym wszystko dąży do spodlenia i wytępienia rasy. Jedni giną z niedostatku, drudzy z rozpusty. Praca odejmuje sobie od ust, ażeby karmić niedołęgów; miłosierdzie hoduje bezczelnych próżniaków, a ubóstwo nie mogące zdobyć się na sprzęty otacza się wiecznie głodnymi dziećmi, których największą zaletą jest wczesna śmierć.
Tu nie poradzi jednostka z inicjatywą, bo wszystko sprzysięgło się, ażeby ją spętać i zużyć w pustej walce — o nic.”

Gdy Wokulski zadzwonił do mieszkania lekarza, ten właśnie był zajęty gatunkowaniem włosów rozmaitych osobników rasy słowiańskiej, germańskiej i semickiej i przy pomocy mikroskopu mierzył dłuższe i krótsze średnice ich przekrojów.

— A, jesteś?… — rzekł do Wokulskiego odwracając głowę. — Nałóż sobie fajkę, jeżeli chcesz, i kładź się na kanapie, jeżeli się zmieścisz.

Gość zapalił fajkę i położył się, jak mu kazano, doktór robił swoje. Przez pewien czas obaj milczeli, wreszcie odezwał się Wokulski:

— Powiedz mi: czy medycyna zna taki stan umysłu, w którym człowiekowi wydaje się, że jego rozproszone dotychczas wiadomości i… uczucia złączyły się jakby w jeden organizm?
— Owszem. Przy ciągłej pracy umysłowej i dobrym odżywianiu mogą wytworzyć się w mózgu nowe komórki albo — skojarzyć się między sobą dawne. No i wówczas z rozmaitych departamentów

mózgu i z rozmaitych dziedzin wiedzy tworzy się jedna całość.

— A co znaczy taki stan umysłu, w którym człowiek obojętnieje dla śmierci, ale za to poczyna tęsknić do legend o życiu wiecznym?…

— Obojętność dla śmierci — odpowiedział doktór — jest cechą umysłów dojrzałych, a pociąg do życia wiecznego — zapowiedzią nadchodzącej starości.

Znowu umilkli. Gość palił fajkę, gospodarz kręcił się nad mikroskopem.

— Czy myślisz — spytał Wokulski — że można… kochać kobietę w sposób idealny, nie pożądając jej?

— Naturalnie. Jest to jedna z masek, w którą lubi przebierać się instynkt utrwalenia gatunku.

— Instynkt — gatunek — instynkt utrwalenia czegoś i — utrwalenie gatunku!… — powtórzył Wokulski. — Trzy wyrazy, a cztery głupstwa.

— Zrób szóste — odpowiedział doktór nie odejmując oka od szkła — i ożeń się.

— Szóste?… — rzekł Wokulski podnosząc się na kanapie. — A gdzież piąte?

— Piąte już zrobiłeś: zakochałeś się.

— Ja?… W moim wieku?…

— Czterdzieści pięć lat — to epoka ostatniej miłości, najgorszej — odpowiedział doktór.

— Znawcy mówią, że pierwsza miłość jest najgorsza — szepnął Wokulski.
— Nieprawda. Po pierwszej czeka cię sto innych, ale po setnej pierwszej — już nic. Żeń się; jedyny to ratunek na twoją chorobę.
"""

In [3]:
inputs = tokenizer(sample_text, padding=True, truncation=True, return_tensors="pt")

#### Inference time for different inference modes

In [4]:
import time
import torch

- No optimization

In [5]:
start = time.time()

for _ in range(100):
    outputs = model(**inputs)

end = time.time()

res_no_opt = (end - start) / 100
res_no_opt

1.2029865765571595

- eval mode

In [6]:
start = time.time()

model.eval()
for _ in range(100):
    outputs = model(**inputs)

end = time.time()

res_eval = (end - start) / 100
res_eval

1.1169808459281922

In [7]:
start = time.time()

model.eval()

with torch.no_grad():
    for _ in range(100):
        outputs = model(**inputs)

end = time.time()

res_no_grad = (end - start) / 100
res_no_grad

1.0992486834526063

In [8]:
start = time.time()

model.eval()

with torch.inference_mode():
    for _ in range(100):
        outputs = model(**inputs)

end = time.time()
res_infer = (end - start) / 100
res_infer

1.1085404634475708

Speedup

In [9]:
print(f"{' inference mode':35} | speedup (baseline/mode)")
print("-" * 62)
print(f"{'model.eval()':35} | {res_no_opt / res_eval:.2f}x")
print(f"{'model.eval() + torch.no_grad()':35} | {res_no_opt / res_no_grad:.2f}x")
print(f"{'model.eval() + inference_mode()':35} | {res_no_opt / res_infer:.2f}x")

 inference mode                     | speedup (baseline/mode)
--------------------------------------------------------------
model.eval()                        | 1.08x
model.eval() + torch.no_grad()      | 1.09x
model.eval() + inference_mode()     | 1.09x


**Comment:** The results are slightly better for subsequent inference modes as compared to the non-optimised inference mode.

## PyTorch model compilation - Exercise 2

In [10]:
start = time.time()

model.eval()
compiled_model = torch.compile(model)

# single inference
compiled_model.eval()

with torch.inference_mode():
    outputs = compiled_model(**inputs)

end = time.time()

comp_and_warmup = end - start
print(f"Compilation and inference warm-up time: {comp_and_warmup:.2f}")

Compilation and inference warm-up time: 38.24


In [11]:
start = time.time()

# compiled_model.eval()

with torch.inference_mode():
    for _ in range(100):
        outputs = compiled_model(**inputs)

end = time.time()
res_comp_infer = (end - start) / 100

print(f"Compiled model average inference time in inference mode: {res_comp_infer:.2f}")
print(f"Compiled model speedup in inference mode: {res_no_opt / res_comp_infer:.2f}x")

Compiled model average inference time in inference mode: 1.43
Compiled model speedup in inference mode: 0.84x


**Comment:** For this task model compilaton did not improve inference time.

## Quantization - Exercise 3

"Another way to optimize a model is to quantize its weights, reducing its size, but also the precision."

In [12]:
model = model.to("cpu")

In [13]:
model_quantized = torch.ao.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

/tmp/ipython-input-2605189210.py:1: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_quantized = torch.ao.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


In [14]:
print(model_quantized)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (o): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (dropout): Dropout(p=0.1, inplace=F

In [15]:
import os

os.getcwd()

'/content'

In [16]:
# saving both models
torch.save(model.state_dict(), "model.pt")
torch.save(model_quantized.state_dict(), "model_quantized.pt")

Model size comparison

In [17]:
model_size_mb = os.path.getsize("model.pt") / (1024**2)
model_quantized_size_mb = os.path.getsize("model_quantized.pt") / (1024**2)

print(f"Size of normal model: {model_size_mb:.2f} MB")
print(f"Size of quantized model: {model_quantized_size_mb:.2f} MB")

Size of normal model: 417.72 MB
Size of quantized model: 173.10 MB


**Comment:** The quantized model takes up over two times less memory than the non-quantized model.

Speedup comparison

In [18]:
start = time.time()

model.eval()

with torch.inference_mode():
    for _ in range(100):
        outputs = model(**inputs)

end = time.time()

res_origin_infer = (end - start) / 100
res_origin_infer

1.0660580229759216

In [19]:
start = time.time()

model_quantized.eval()

with torch.inference_mode():
    for _ in range(100):
        outputs = model_quantized(**inputs)

end = time.time()
res_quant_infer = (end - start) / 100

print(
    f"Quantized model average inference time in inference mode: {res_quant_infer:.2f}"
)
print(
    f"Quantized model speedup in inference mode: {res_origin_infer / res_quant_infer:.2f}x"
)

Quantized model average inference time in inference mode: 0.87
Quantized model speedup in inference mode: 1.22x


**Comment:** Quantization also allowed for a substantial speedup comapred to the non-quantized model (both used in the inference_mode mode).

## GPU optimization strategies - Exercise 4

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
model_gpu = model.to(device)
inputs_gpu = {k: v.to(device) for k, v in inputs.items()}

In [22]:
model_gpu.eval()
compiled_model_gpu = torch.compile(model_gpu)

compiled_model_gpu.eval()
with torch.inference_mode():
    outputs = compiled_model_gpu(**inputs_gpu)

/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
W0105 21:42:26.078000 287 torch/_inductor/utils.py:1558] [0/1] Not enough SMs to use max_autotune_gemm mode


In [23]:
# Enable CUDA Graphs for maximum throughput
model_gpu.eval()
compiled_model_with_cudagraphs = torch.compile(model_gpu, mode="max-autotune")

compiled_model_with_cudagraphs.eval()
with torch.inference_mode():
    outputs = compiled_model_with_cudagraphs(**inputs_gpu)

Autotune Choices Stats:
{"num_choices": 2, "num_triton_choices": 0, "best_kernel": "bias_addmm", "best_time": 0.27852800488471985}
AUTOTUNE addmm(512x768, 512x768, 768x768)
strides: [0, 1], [768, 1], [1, 768]
dtypes: torch.float32, torch.float32, torch.float32
  bias_addmm 0.2785 ms 100.0% 
  addmm 0.2908 ms 95.8% 
SingleProcess AUTOTUNE benchmarking takes 0.0987 seconds and 0.0002 seconds precompiling for 2 choices


In [24]:
# Enable max-autotune without CUDA Graphs for dynamic input shapes
model_gpu.eval()
compiled_model_dynamic = torch.compile(model_gpu, mode="max-autotune-no-cudagraphs")

compiled_model_dynamic.eval()
with torch.inference_mode():
    outputs = compiled_model_dynamic(**inputs_gpu)

Inference time comparison for different compile modes and input sizes

In [ ]:
def measure_inference_time(model, inputs, n_runs):
    start = time.time()

    model.eval()

    with torch.inference_mode():
        for _ in range(n_runs):
            _ = model(**inputs)

    if device == "cuda":
        torch.cuda.synchronize()  # wait for GPU to finish work

    end = time.time()

    return (end - start) / n_runs

In [26]:
res_comp_gpu = measure_inference_time(compiled_model_gpu, inputs_gpu, 1000)
res_comp_gpu

0.03434568333625793

In [27]:
res_comp_cuda_gpu = measure_inference_time(
    compiled_model_with_cudagraphs, inputs_gpu, 1000
)
res_comp_cuda_gpu

0.035183266878128054

In [28]:
res_comp_dynamic_gpu = measure_inference_time(compiled_model_dynamic, inputs_gpu, 1000)
res_comp_dynamic_gpu

0.03412899374961853

In [41]:
print(f"{'inference mode':65} | {'time':10} | speedup (baseline/mode)")
print("-" * 100)
print(f"{'torch.compile()':65} | {res_comp_gpu:10.2f} | -")
print(
    f"{'torch.compile() with mode="max-autotune"':65} | {res_comp_cuda_gpu:10.2f} | {res_comp_gpu / res_comp_cuda_gpu:.2f}x"
)
print(
    f"{'torch.compile() with mode="max-autotune-no-cudagraphs"':65} | {res_comp_dynamic_gpu:10.2f} | {res_comp_gpu / res_comp_dynamic_gpu:.2f}x"
)

inference mode                                                    | time       | speedup (baseline/mode)
----------------------------------------------------------------------------------------------------
torch.compile()                                                   |       0.03 | -
torch.compile() with mode="max-autotune"                          |       0.04 | 0.98x
torch.compile() with mode="max-autotune-no-cudagraphs"            |       0.03 | 1.01x


**Comment:** The speedup for the graph optimized modes is not significant for the current input, possibly due to the overhead related to transferring data to the GPU?

 "CUDA Graphs become invalid when input dimensions change. - They are static by design."
 - "The `max-autotune-no-cudagraphs` mode addresses this limitation."

In [29]:
# 5x shorter input
short_text = sample_text[: len(sample_text) // 5]

short_inputs = tokenizer(short_text, padding=True, truncation=True, return_tensors="pt")
short_inputs_gpu = {k: v.to(device) for k, v in short_inputs.items()}

# a couple inference warm-up runs for each model
measure_inference_time(compiled_model_gpu, short_inputs_gpu, 10)
measure_inference_time(compiled_model_with_cudagraphs, short_inputs_gpu, 10)
measure_inference_time(compiled_model_dynamic, short_inputs_gpu, 10)

Autotune Choices Stats:
{"num_choices": 2, "num_triton_choices": 0, "best_kernel": "bias_addmm", "best_time": 0.2131199985742569}
AUTOTUNE addmm(347x768, 347x768, 768x768)
strides: [0, 1], [768, 1], [1, 768]
dtypes: torch.float32, torch.float32, torch.float32
  bias_addmm 0.2131 ms 100.0% 
  addmm 0.2273 ms 93.8% 
SingleProcess AUTOTUNE benchmarking takes 0.0660 seconds and 0.0003 seconds precompiling for 2 choices


2.921022152900696

In [30]:
res_comp_gpu_short = measure_inference_time(compiled_model_gpu, short_inputs_gpu, 1000)

res_comp_cuda_gpu_short = measure_inference_time(
    compiled_model_with_cudagraphs, short_inputs_gpu, 1000
)

res_comp_dynamic_gpu_short = measure_inference_time(
    compiled_model_dynamic, short_inputs_gpu, 1000
)

print(f"{'inference mode':65} | {'time':10} | speedup (baseline/mode)")
print("-" * 100)
print(f"{'torch.compile()':65} | {res_comp_gpu_short:10.2f} | -")
print(
    f"{'torch.compile() with mode="max-autotune"':65} | {res_comp_cuda_gpu_short:10.2f} | {res_comp_gpu_short / res_comp_cuda_gpu_short:.2f}x"
)
print(
    f"{'torch.compile() with mode="max-autotune-no-cudagraphs"':65} | {res_comp_dynamic_gpu_short:10.2f} | {res_comp_gpu_short / res_comp_dynamic_gpu_short:.2f}x"
)

inference mode                                                    | time       | speedup (baseline/mode)
----------------------------------------------------------------------------------------------------
torch.compile()                                                   |       0.03 | -
torch.compile() with mode="max-autotune"                          |       0.03 | 1.06x
torch.compile() with mode="max-autotune-no-cudagraphs"            |       0.03 | 1.07x


**Comment:** For shorter input there seems to be a slight speedup for CUDA graphs compared to the torch compiled-only model.

In [31]:
# 5x longer input
long_text = sample_text * 5

long_inputs = tokenizer(long_text, padding=True, truncation=True, return_tensors="pt")
long_inputs_gpu = {k: v.to(device) for k, v in long_inputs.items()}

measure_inference_time(compiled_model_gpu, long_inputs_gpu, 10)
measure_inference_time(compiled_model_with_cudagraphs, long_inputs_gpu, 10)
measure_inference_time(compiled_model_dynamic, long_inputs_gpu, 10)

0.04739067554473877

In [32]:
res_comp_gpu_long = measure_inference_time(compiled_model_gpu, long_inputs_gpu, 1000)

res_comp_cuda_gpu_long = measure_inference_time(
    compiled_model_with_cudagraphs, long_inputs_gpu, 1000
)

res_comp_dynamic_gpu_long = measure_inference_time(
    compiled_model_dynamic, long_inputs_gpu, 1000
)

print(f"{'inference mode':65} | {'time':10} | speedup (baseline/mode)")
print("-" * 100)
print(f"{'torch.compile()':65} | {res_comp_gpu_long:10.2f} | -")
print(
    f"{'torch.compile() with mode="max-autotune"':65} | {res_comp_cuda_gpu_long:10.2f} | {res_comp_gpu_long / res_comp_cuda_gpu_long:.2f}x"
)
print(
    f"{'torch.compile() with mode="max-autotune-no-cudagraphs"':65} | {res_comp_dynamic_gpu_long:10.2f} | {res_comp_gpu_long / res_comp_dynamic_gpu_long:.2f}x"
)

inference mode                                                    | time       | speedup (baseline/mode)
----------------------------------------------------------------------------------------------------
torch.compile()                                                   |       0.04 | -
torch.compile() with mode="max-autotune"                          |       0.04 | 1.00x
torch.compile() with mode="max-autotune-no-cudagraphs"            |       0.04 | 0.99x


**Comment:** For a longer input there seems to be no speedup compared to the compiled model. The inference is slightly longer than for the shorter inputs which is expected.

## Changing numerical precision - Exercise 5

In [33]:
# checking if NVidia GPU supports fast float16 (via Tensor Cores)
import torch

capability = torch.cuda.get_device_capability()
print(f"CUDA device capability: {capability}")

# Tensor Cores are available on NVidia GPUs with CUDA >= 7 (e.g. Volta, Turing, Ampere, Hopper)
if capability >= (7, 0):
    print("Tensor Cores available: fast float16 supported.")
else:
    print("Tensor Cores not available: float16 may be slow or unsupported.")

CUDA device capability: (7, 5)
Tensor Cores available: fast float16 supported.


In [34]:
model = AutoModel.from_pretrained(model_name).to("cuda")
print(f"Data type of model parameters: {model.dtype}")

Data type of model parameters: torch.float32


In [35]:
# casting model weights and inputs to half-precision:
model_half = model.half().to("cuda")

input_ids = inputs_gpu["input_ids"]
attention_mask = inputs_gpu["attention_mask"]

# this didn't work
# input_ids_half = input_ids.to('cuda').half()
# attention_half = attention_mask.to('cuda').half()

outputs = model_half(**inputs_gpu)

In [36]:
# verify
print(f"Data type of model parameters: {model.dtype}")
print(f"Data type of inputs: {input_ids.dtype}, {attention_mask.dtype}")
print(f"Data type of model_half parameters: {model_half.dtype}")

Data type of model parameters: torch.float16
Data type of inputs: torch.int64, torch.int64
Data type of model_half parameters: torch.float16


- `model` also gets altered by running `.half()`

In [37]:
# reinitalise the model bcs model.half works inplace
model = AutoModel.from_pretrained(model_name).to("cuda")
print(f"Data type of model parameters: {model.dtype}")
print(f"Data type of model_half parameters: {model_half.dtype}")

Data type of model parameters: torch.float32
Data type of model_half parameters: torch.float16


Inference time comparison

In [ ]:
def measure_autocast_inference_time(model, inputs, n_runs):
    start = time.time()

    model.eval()

    with torch.inference_mode():
        with torch.autocast(device_type="cuda", dtype=torch.float16):
            for _ in range(n_runs):
                _ = model(**inputs)

    if device == "cuda":
        torch.cuda.synchronize()

    end = time.time()

    return (end - start) / n_runs

In [39]:
# warm-up runs
measure_inference_time(model, inputs_gpu, 10)
measure_inference_time(model_half, inputs_gpu, 10)
measure_autocast_inference_time(model, inputs_gpu, 10)

0.0147691011428833

In [40]:
res_model32 = measure_inference_time(model, inputs_gpu, 1000)
res_model_half = measure_inference_time(model_half, inputs_gpu, 1000)
res_model_autocast = measure_autocast_inference_time(model, inputs_gpu, 1000)

print(f"{'model':65} | {'time':10} | speedup (baseline/mode)")
print("-" * 100)
print(f"{'float32 model':65} | {res_model32:10.2f} | -")
print(
    f"{'float16 model':65} | {res_model_half:10.2f} | {res_model32 / res_model_half:.2f}x"
)
print(
    f"{'float32 model with autocast':65} | {res_model_autocast:10.2f} | {res_model32 / res_model_autocast:.2f}x"
)

model                                                             | time       | speedup (baseline/mode)
----------------------------------------------------------------------------------------------------
float32 model                                                     |       0.04 | -
float16 model                                                     |       0.01 | 3.15x
float32 model with autocast                                       |       0.02 | 2.53x


**Comment:**
- The static half-precision (16float) model on GPU allows for a speedup of ~3x as compared to inference using the 32float precision model on GPU. This makes it the largest improvement observed in this lab.
- The autocast model allows for a slightly lower speedup of ~2.5x, so slightly lower than the static cast. However, there is a trade-off, bacuse autocast allows for mixed precision operations, which might allow for better accuracy/performance since more precise operations are done using higher precision.

## ONNX - Exercise 6